In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


READING ALL DATASETS

In [148]:
circuits = pd.read_csv('./data/circuits.csv')
constructor_results = pd.read_csv('./data/constructor_results.csv')
constructor_standings = pd.read_csv('./data/constructor_standings.csv')
constructors = pd.read_csv('./data/constructors.csv')
driver_standings = pd.read_csv('./data/driver_standings.csv')
drivers = pd.read_csv('./data/drivers.csv')
lap_times = pd.read_csv('./data/lap_times.csv')
pit_stops = pd.read_csv('./data/pit_stops.csv')
qualifying = pd.read_csv('./data/qualifying.csv')
races = pd.read_csv('./data/races.csv')
results = pd.read_csv('./data/results.csv')
seasons = pd.read_csv('./data/seasons.csv')
sprint_results = pd.read_csv('./data/sprint_results.csv')
status = pd.read_csv('./data/status.csv')
weather = pd.read_csv('./data/weather.csv')

DROPPING REDUNDANT COLUMNS

In [149]:
del circuits["lat"]
del circuits["lng"]
del circuits["country"]
del circuits["alt"]
del circuits["url"]
del circuits["name"]
del constructor_results['status']
del constructor_standings["positionText"]
del constructors["name"]
del constructors["url"]
driver_standings.drop(['positionText'], axis=1, inplace=True)
drivers.drop(['surname', 'url'], axis=1, inplace=True)
lap_times.drop(['time'], axis=1, inplace=True)
pit_stops.drop(['time', 'duration'], axis=1, inplace=True)
races = races.drop(['url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'], axis = 1)
results.drop(['time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'positionText', 'position'], axis = 1, inplace = True)
seasons.drop(['url'], axis = 1, inplace=True)
weather.drop(['weather'], axis = 1, inplace=True)



RENAMING

In [150]:
weather.rename(columns={'season': 'year'}, inplace = True)

MERGING DATA :(

In [167]:
df_1 = pd.merge(races, weather, on = ['year', 'round'], how = 'outer')
df_1.head()
df_1.shape

(1079, 13)

In [168]:
results.head()
results.shape

(25660, 10)

In [175]:
df_2 = pd.merge(df_1, results, on = 'raceId')
df_2.shape
df_2.head()

,raceId,year,round,circuitId,name,date,time,circuit_id,weather_warm,weather_cold,...,weather_cloudy,resultId,driverId,constructorId,number,grid,positionOrder,points,laps,statusId
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,0.0,7554,18,23,22,1,1,10.0,58,1
1,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,0.0,7555,22,23,23,2,2,8.0,58,1
2,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,0.0,7556,15,7,9,20,3,6.0,58,1
3,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,0.0,7557,10,7,10,19,4,5.0,58,1
4,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,0.0,7558,4,4,7,10,5,4.0,58,1


In [176]:
status.shape

(139, 2)

In [180]:
df_3 = pd.merge(df_2, status, on = 'statusId')
df_3.head()

,raceId,year,round,circuitId,name,date,time,circuit_id,weather_warm,weather_cold,...,resultId,driverId,constructorId,number,grid,positionOrder,points,laps,statusId,status
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,7554,18,23,22,1,1,10.0,58,1,Finished
1,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,7555,22,23,23,2,2,8.0,58,1,Finished
2,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,7556,15,7,9,20,3,6.0,58,1,Finished
3,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,7557,10,7,10,19,4,5.0,58,1,Finished
4,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,7558,4,4,7,10,5,4.0,58,1,Finished


In [182]:
circuits.head()
circuits.shape

(76, 3)

In [194]:
df_4 = pd.merge(df_3, circuits, on = 'circuitId')
df_4.drop(['circuitRef'], axis = 1, inplace=True)
df_4.head()

,raceId,year,round,circuitId,name,date,time,circuit_id,weather_warm,weather_cold,...,driverId,constructorId,number,grid,positionOrder,points,laps,statusId,status,location
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,18,23,22,1,1,10.0,58,1,Finished,Melbourne
1,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,22,23,23,2,2,8.0,58,1,Finished,Melbourne
2,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,15,7,9,20,3,6.0,58,1,Finished,Melbourne
3,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,10,7,10,19,4,5.0,58,1,Finished,Melbourne
4,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,1.0,0.0,...,4,4,7,10,5,4.0,58,1,Finished,Melbourne
